In [6]:
import string
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [7]:
train_data = pd.read_csv("text_emotion.csv")

In [8]:
traindata=train_data.drop(['tweet_id', 'author'], axis=1)

In [9]:
traindata.head(10)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
5,worry,Re-pinging @ghostridah14: why didn't you go to...
6,sadness,"I should be sleep, but im not! thinking about ..."
7,worry,Hmmm. http://www.djhero.com/ is down
8,sadness,@charviray Charlene my love. I miss you
9,sadness,@kelcouch I'm sorry at least it's Friday?


In [10]:
def cleaning(content):
  lower_case = content.lower()
  lower_case.replace("."," ")
  cleaned_text = lower_case.translate(str.maketrans('.',' ',string.punctuation))
  tokenized_words = word_tokenize(cleaned_text,"english")
  final_words = [word for word in tokenized_words if word not in stopwords.words("english")]
  clean_words=[]
  for word in final_words:
    clean_words.append(cons_dupl(word))
  postagged_words=nltk.pos_tag(clean_words)
  lemma_words=[]
  for word,y in postagged_words:
      lemma_words.append(WordNetLemmatizer().lemmatize(word))
  lemma_words = ls(lemma_words)
  return lemma_words

def cons_dupl(word):
     s=''
     for char in word:
       if s=='' or char!=s[len(s)-1]:
         s=s+char
     return s
def ls(content):
  listtostr = ""
  for word in content:
    listtostr +=" "+word
  return listtostr

  

In [11]:
traindata['cleaned_text'] = traindata['content'].apply(cleaning)


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,r2_score,accuracy_score

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
x_features = traindata['cleaned_text']
y_labels = traindata['sentiment']

In [15]:
cv = CountVectorizer()
X = cv.fit_transform(x_features)

In [16]:
x_train,x_test,y_train,y_test=train_test_split(X,y_labels,test_size=0.3,random_state=42)

In [41]:
nv_model = MultinomialNB()
nv_model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [22]:
nv_model.score(x_test,y_test)

0.315

In [23]:
predictvalue = nv_model.predict(x_test)

In [36]:
predictvalue

array(['happiness', 'neutral', 'worry', ..., 'worry', 'neutral', 'worry'],
      dtype='<U10')

In [57]:
import numpy as np

In [116]:
def callmodel1(text):
   vect = cv.transform(text).toarray()
   emotion = nv_model.predict(vect)
   pred_proba =nv_model.predict_proba(vect)
   max=np.max(pred_proba)
   pred_percent_for_all = dict(zip(nv_model.classes_,pred_proba[0]*100))
   print(pred_percent_for_all)
   return emotion,max,pred_percent_for_all

In [118]:
text = input("text here")
in_text = []
in_text.append(text)
output=callmodel1(in_text)
emotion = output[0]
statement=output[2]
maxim = output[1]
x = list(statement.keys())
y = list(statement.values())
print("emotion: ")
print(emotion)
print("strength:")
print(maxim)
plt.bar(x,y)
plt.show()

text herei love india and i love python
{'anger': 5.573457197676421e-05, 'boredom': 0.0001924056139486012, 'empty': 0.002768853667691148, 'enthusiasm': 0.022401576132908788, 'fun': 0.09710910151285021, 'happiness': 11.756126615874058, 'hate': 0.017074664488794283, 'love': 73.2036518327141, 'neutral': 3.3680428270597735, 'relief': 0.0867891430785499, 'sadness': 8.575230387449457, 'surprise': 0.49198241221918154, 'worry': 2.3785744456170375}
emotion: 
['love']
strength:
0.732036518327141
